In [78]:
from functools import reduce

import math
from neo4j import GraphDatabase
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import udf
from pyspark.sql import types as T
import pandas as pd
from ydata_profiling import ProfileReport
import scipy.stats as st

In [2]:
url = "neo4j+s://e13d26eb.databases.neo4j.io"
username = "neo4j"
password = "C0-lgeL4HXJTknM9OI0qq-2mGvZ5hI41521JX9WvqO0"

In [3]:
spark = (
    SparkSession.builder.config("neo4j.url", url)
    .config("neo4j.authentication.type", "basic")
    .config('spark.jars.packages', 'org.neo4j:neo4j-connector-apache-spark_2.12:5.1.0_for_spark_3')
    .config("neo4j.authentication.basic.username", username)
    .config("neo4j.authentication.basic.password", password)
    .getOrCreate()
)

24/06/03 20:05:11 WARN Utils: Your hostname, emil-GL552VW resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp2s0)
24/06/03 20:05:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/emil/spark/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/emil/.ivy2/cache
The jars for the packages stored in: /home/emil/.ivy2/jars
org.neo4j#neo4j-connector-apache-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2ebb33e4-6009-4c0b-a73d-6db40be47832;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.12;5.1.0_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.12_common;5.1.0 in central
	found org.neo4j.driver#neo4j-java-driver;4.4.12 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found org.apache.xbean#xbean-asm6-shaded;4.10 in central
	found org.neo4j#neo4j-cypher-dsl;2022.9.0 in central
	found org.apiguardian#apiguardian-api;1.1.2 in central
:: resolution report :: resolve 362ms :: artifacts dl 15ms
	:: modules in use:
	org.apache.xbean#xbean-asm6-shaded;4.10 from central in [default]
	org.apiguardian#apiguardian-api;1.1.2 from central in [default]
	org.neo4j#neo4j-connector-apache-spark_2.12;5

24/06/03 20:05:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/06/03 20:05:12 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
spark

# Load data

In [20]:
movilens_path = "../data/ml-latest-small/"
imdb_path = "../data/imdb/"

In [21]:
raw_movies = spark.read.option("header", True).format("csv").load(movilens_path + "movies.csv")
raw_ratings = spark.read.option("header", True).format("csv").load(movilens_path + "ratings.csv")
raw_links = spark.read.option("header", True).format("csv").load(movilens_path + "links.csv")
raw_tags = spark.read.option("header", True).format("csv").load(movilens_path + "tags.csv")

In [22]:
raw_title_basics = spark.read.option("header", True).option("delimiter", "\t").format("csv").load(imdb_path + "title.basics.tsv")
raw_title_crew = spark.read.option("header", True).format("csv").option("delimiter", "\t").load(imdb_path + "title.crew.tsv")
raw_name = spark.read.option("header", True).format("csv").option("delimiter", "\t").load(imdb_path + "name.basics.tsv")
raw_principals = spark.read.option("header", True).format("csv").option("delimiter", "\t").load(imdb_path + "title.principals.tsv")

# EDA

In [18]:
raw_movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [80]:
raw_movies.count()

9742

In [22]:
raw_ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [10]:
raw_ratings.count()

100836

In [23]:
raw_links.show()

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
|      6|0113277|   949|
|      7|0114319| 11860|
|      8|0112302| 45325|
|      9|0114576|  9091|
|     10|0113189|   710|
|     11|0112346|  9087|
|     12|0112896| 12110|
|     13|0112453| 21032|
|     14|0113987| 10858|
|     15|0112760|  1408|
|     16|0112641|   524|
|     17|0114388|  4584|
|     18|0113101|     5|
|     19|0112281|  9273|
|     20|0113845| 11517|
+-------+-------+------+
only showing top 20 rows



In [25]:
raw_tags.show()

+------+-------+-----------------+----------+
|userId|movieId|              tag| timestamp|
+------+-------+-----------------+----------+
|     2|  60756|            funny|1445714994|
|     2|  60756|  Highly quotable|1445714996|
|     2|  60756|     will ferrell|1445714992|
|     2|  89774|     Boxing story|1445715207|
|     2|  89774|              MMA|1445715200|
|     2|  89774|        Tom Hardy|1445715205|
|     2| 106782|            drugs|1445715054|
|     2| 106782|Leonardo DiCaprio|1445715051|
|     2| 106782|  Martin Scorsese|1445715056|
|     7|  48516|     way too long|1169687325|
|    18|    431|        Al Pacino|1462138765|
|    18|    431|         gangster|1462138749|
|    18|    431|            mafia|1462138755|
|    18|   1221|        Al Pacino|1461699306|
|    18|   1221|            Mafia|1461699303|
|    18|   5995|        holocaust|1455735472|
|    18|   5995|       true story|1455735479|
|    18|  44665|     twist ending|1456948283|
|    18|  52604|  Anthony Hopkins|

In [39]:
raw_title_basics.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [43]:
raw_title_crew.show()

+---------+-------------------+---------+
|   tconst|          directors|  writers|
+---------+-------------------+---------+
|tt0000001|          nm0005690|       \N|
|tt0000002|          nm0721526|       \N|
|tt0000003|          nm0721526|       \N|
|tt0000004|          nm0721526|       \N|
|tt0000005|          nm0005690|       \N|
|tt0000006|          nm0005690|       \N|
|tt0000007|nm0005690,nm0374658|       \N|
|tt0000008|          nm0005690|       \N|
|tt0000009|          nm0085156|nm0085156|
|tt0000010|          nm0525910|       \N|
|tt0000011|          nm0804434|       \N|
|tt0000012|nm0525908,nm0525910|       \N|
|tt0000013|          nm0525910|       \N|
|tt0000014|          nm0525910|       \N|
|tt0000015|          nm0721526|       \N|
|tt0000016|          nm0525910|       \N|
|tt0000017|nm1587194,nm0804434|       \N|
|tt0000018|          nm0804434|       \N|
|tt0000019|          nm0932055|       \N|
|tt0000020|          nm0010291|       \N|
+---------+-------------------+---

In [45]:
raw_name.show()

+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|actor,miscellaneo...|tt0072308,tt00504...|
|nm0000002|      Lauren Bacall|     1924|     2014|actress,soundtrac...|tt0037382,tt00752...|
|nm0000003|    Brigitte Bardot|     1934|       \N|actress,music_dep...|tt0057345,tt00491...|
|nm0000004|       John Belushi|     1949|     1982|actor,writer,musi...|tt0072562,tt00779...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00839...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,producer,...|tt0034583,tt00368...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,producer,mi...|tt0034583,tt00425...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,dir

In [23]:
raw_principals.show(truncate=False)

+---------+--------+---------+---------------+-----------------------+--------------+
|tconst   |ordering|nconst   |category       |job                    |characters    |
+---------+--------+---------+---------------+-----------------------+--------------+
|tt0000001|1       |nm1588970|self           |\N                     |["Self"]      |
|tt0000001|2       |nm0005690|director       |\N                     |\N            |
|tt0000001|3       |nm0005690|producer       |producer               |\N            |
|tt0000001|4       |nm0374658|cinematographer|director of photography|\N            |
|tt0000002|1       |nm0721526|director       |\N                     |\N            |
|tt0000002|2       |nm1335271|composer       |\N                     |\N            |
|tt0000003|1       |nm0721526|director       |\N                     |\N            |
|tt0000003|2       |nm1770680|producer       |producer               |\N            |
|tt0000003|3       |nm0721526|producer       |producer

# Generate primary movies dataset

In [7]:
directors = raw_title_crew.join(raw_name, raw_title_crew.directors==raw_name.nconst, how="inner")

In [8]:
directors.show()

+----------+---------+--------------------+---------+--------------+---------+---------+--------------------+--------------------+
|    tconst|directors|             writers|   nconst|   primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+----------+---------+--------------------+---------+--------------+---------+---------+--------------------+--------------------+
| tt0284008|nm0000100|nm0001037,nm00000...|nm0000100|Rowan Atkinson|     1955|       \N|actor,writer,prod...|tt0274166,tt16341...|
| tt0119600|nm0000113|           nm0000113|nm0000113|Sandra Bullock|     1964|       \N|producer,actress,...|tt1041829,tt02123...|
| tt0328097|nm0000161| nm0853256,nm0178294|nm0000161|   Salma Hayek|     1966|       \N|actress,producer,...|tt0120679,tt01128...|
| tt1192878|nm0000174|           nm0000174|nm0000174|    Val Kilmer|     1959|       \N|actor,producer,di...|tt0101761,tt01200...|
| tt6857412|nm0000174|           nm0000174|nm0000174|    Val Kilmer|     1959|     

In [10]:
links = raw_links.select(F.col("movieId"), F.concat(F.lit("tt"), F.col("imdbId")).alias("tconst"))

In [11]:
movies = directors.join(links, on="tconst", how="inner").join(raw_movies, on="movieId", how="inner")

In [ ]:
movies = movies.select("*", F.split("genres", r"\|").alias("genres_array")).select(
    F.col("movieId").alias("movie_id"),
    "title",
    F.explode("genres_array").alias("genres"),
    F.col("primaryName").alias("director_name"),
    "tconst"
)

In [6]:
movies.count()

19923

### Write primary movies dataset

In [ ]:
movies.write.option("header", True).csv("../data/results/prm_movies")

In [5]:
movies = spark.read.option("header", True).format("csv").load("../data/results/" + "prm_movies")

# Generate master graph data table

In [23]:
graph_mdt = movies.join(
    raw_ratings.select(
        F.col("userId").alias("user_id"),
        F.col("movieId").alias("movie_id"),
        "rating"
    ),
    on="movie_id",
    how="inner"
)

In [26]:
graph_mdt.show()

+--------+--------------------+---------+----------------+-------+------+
|movie_id|               title|   genres|   director_name|user_id|rating|
+--------+--------------------+---------+----------------+-------+------+
|       1|    Toy Story (1995)|  Fantasy|   John Lasseter|      1|   4.0|
|       1|    Toy Story (1995)|   Comedy|   John Lasseter|      1|   4.0|
|       1|    Toy Story (1995)| Children|   John Lasseter|      1|   4.0|
|       1|    Toy Story (1995)|Animation|   John Lasseter|      1|   4.0|
|       1|    Toy Story (1995)|Adventure|   John Lasseter|      1|   4.0|
|       3|Grumpier Old Men ...|  Romance|   Howard Deutch|      1|   4.0|
|       3|Grumpier Old Men ...|   Comedy|   Howard Deutch|      1|   4.0|
|       6|         Heat (1995)| Thriller|    Michael Mann|      1|   4.0|
|       6|         Heat (1995)|    Crime|    Michael Mann|      1|   4.0|
|       6|         Heat (1995)|   Action|    Michael Mann|      1|   4.0|
|      47|Seven (a.k.a. Se7...| Thrill

# Profiling data

In [99]:
graph_mdt_pd = graph_mdt.toPandas()

In [11]:
# Generate the data profiling report 
report = ProfileReport(graph_mdt_pd, title='MovieLens dataset')
report.to_file("my_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/emil/anaconda3/envs/RS-T2/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'Fantasy'')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Write to Neo4J

In [9]:
graph_mdt = spark.read.option("header", True).format("csv").load("data/results/" + "graph_mdt")

In [ ]:
graph_mdt = graph_mdt.withColumnRenamed("genres", "genre")

In [8]:
graph_mdt.select()

DataFrame[movie_id: string, title: string, genre: string, director_name: string, user_id: string, rating: string]

### Write user nodes

In [4]:

graph_mdt.select("user_id").distinct().write \
    .format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("labels", ":User") \
    .option("node.keys", "user_id") \
    .save()

### Write movies with its director

In [ ]:
graph_mdt.select("movie_id", "director_name").distinct().write \
    .format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("labels", ":Movie") \
    .option("node.keys", "movie_id") \
    .option("node.properties", "director") \
    .save()

### Write genres

In [6]:
graph_mdt.select("genre").distinct().write \
    .format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("labels", ":Genre") \
    .option("node.keys", "genre") \
    .save()

### Generate relationships

In [7]:
graph_mdt.write \
    .format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("relationship", "BELONGS_TO") \
    .option("relationship.save.strategy", "keys") \
    .option("relationship.source.labels", ":Movie") \
    .option("relationship.source.save.mode", "match") \
    .option("relationship.source.node.keys", "movie_id") \
    .option("relationship.target.labels", ":Genre") \
    .option("relationship.target.save.mode", "match") \
    .option("relationship.target.node.keys", "genre") \
    .save()

In [11]:
graph_mdt.select("movie_id", "user_id", F.col("rating").cast("float")).write \
    .format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("relationship", "RATED") \
    .option("relationship.save.strategy", "keys") \
    .option("relationship.source.labels", ":User") \
    .option("relationship.source.save.mode", "match") \
    .option("relationship.source.node.keys", "user_id") \
    .option("relationship.target.labels", ":Movie") \
    .option("relationship.target.save.mode", "match") \
    .option("relationship.target.node.keys", "movie_id") \
    .option("relationship.properties", "rating") \
    .save()

# Generate recomendations for especific users

In [13]:
URI = url
AUTH = (username, password)

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [29]:
user_id = "101"

collaborative_filtering_query = (
    'MATCH (u:User {user_id: "' + user_id + '"})-[s:SIMILARITY]-(other:User)-[r:RATED]->(m:Movie)'
    + """WHERE NOT EXISTS { MATCH (u)-[:RATED]->(m) }
    WITH m, other, toFloat(r.rating) AS otherRating, s.pearsonSimilarity AS similarity
    WITH m, AVG(otherRating * similarity) AS weightedRating, SUM(similarity) AS sumSimilarity
    RETURN m.movie_id, sumSimilarity
    LIMIT 10;"""
)

records, summary, keys = driver.execute_query(
    collaborative_filtering_query,
    database_="neo4j",
)

/tmp/ipykernel_636759/1480557147.py:12: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


In [28]:
records, summary, keys = driver.execute_query(
    "MATCH (p:Person) RETURN p.name AS name",
    database_="neo4j",
)

/tmp/ipykernel_636759/1538916520.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


In [30]:
records

[<Record m.movie_id='3614' sumSimilarity=2.928196250589005>,
 <Record m.movie_id='2279' sumSimilarity=2.3929466346818287>,
 <Record m.movie_id='3255' sumSimilarity=11.097124437173987>,
 <Record m.movie_id='1267' sumSimilarity=9.852849699088182>,
 <Record m.movie_id='3785' sumSimilarity=16.77899487438135>,
 <Record m.movie_id='2133' sumSimilarity=6.294573285313581>,
 <Record m.movie_id='2374' sumSimilarity=2.912493038391483>,
 <Record m.movie_id='1834' sumSimilarity=4.197096829465944>,
 <Record m.movie_id='35836' sumSimilarity=18.055773480346314>,
 <Record m.movie_id='4293' sumSimilarity=1.823615287233232>]

In [23]:
summary

In [24]:
keys

['m.movie_id', 'sumSimilarity']

# Generate recommendations for new users

In [90]:
# It will be defined a function wich calculates a Bayesian Score
# https://www.evanmiller.org/ranking-items-with-star-ratings.html
# so as to compare a raking made by this score and mean score
def bayesian_score(ratings, confidence=0.95):
    """
      Function to calculate wilson score for N star rating system.

      params:
        ratings (list):
          Count of star ratings where ith index represent the votes for that category i.e. [3, 5, 6, 7, 10]
          First value are califications with 1 star, second with 2 and so on...
        confidence (int): Confidence interval
      returns:
        score (float): Bayesian score value
    """
    # If there is no ratings to analize, returns a Beyesian score of zero
    if sum(ratings) == 0:
        return 0

    # Define necessary expressions
    rating_scale = len(ratings)
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    ratings_sum = sum(ratings)
    first_expr = 0.0
    second_expr = 0.0

    # Calculates Bayesian score
    for k in range(len(ratings)):
        first_expr += (k + 1)*(ratings[k] + 1) / (ratings_sum + rating_scale)
        second_expr += (k + 1)*(k + 1)*(ratings[k] + 1) / (ratings_sum + rating_scale)
    score = first_expr - z * math.sqrt(
        (second_expr - first_expr*first_expr) / (ratings_sum + rating_scale + 1)
      )

    return score

In [26]:
ratings = graph_mdt.select("movie_id", "rating")

In [45]:
ratings

DataFrame[movie_id: string, rating: string]

In [31]:
ratings.select("rating").distinct().show()

+------+
|rating|
+------+
|   1.0|
|   4.5|
|   2.5|
|   3.5|
|   5.0|
|   0.5|
|   4.0|
|   1.5|
|   2.0|
|   3.0|
+------+



In [46]:
dfs_by_rating = []

rating_scale = [str(i/2) for i in range(1, 11, 1)]

rating_scale_labels = []

for rating in rating_scale:
  rating_label = rating.replace(".", ",")
  df = ratings.filter(
    f"rating == '{rating}'"
  ).groupBy("movie_id").agg(F.count("rating").alias(f"number_rating_{rating_label}"))
  dfs_by_rating.append(df)
  rating_scale_labels.append(rating_label)

df = reduce(lambda x, y: x.unionByName(y, allowMissingColumns=True), dfs_by_rating)
df = df.fillna(0)

df = df.select(
  "*",
  F.array([f"number_rating_{rating_label}" for rating_label in rating_scale_labels]).alias("list_of_scores")
)

In [47]:
df.show()

+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+--------------------+
|movie_id|number_rating_0,5|number_rating_1,0|number_rating_1,5|number_rating_2,0|number_rating_2,5|number_rating_3,0|number_rating_3,5|number_rating_4,0|number_rating_4,5|number_rating_5,0|      list_of_scores|
+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+--------------------+
|    3210|                3|                0|                0|                0|                0|                0|                0|                0|                0|                0|[3, 0, 0, 0, 0, 0...|
|    4821|                2|                0|                0|                0|                0|                0|                0|                

In [91]:
# @udf("double")
# def bayesian_score_udf(s: pd.Series) -> float:
#     rating =  bayesian_score(list(s))
#     return rating

bayesian_score_udf = udf(
    lambda array_of_ratings: bayesian_score(array_of_ratings), T.LongType()
)

In [86]:
bayesian_score_pd = df.toPandas()

In [101]:
bayesian_score_pd

,movie_id,"number_rating_0,5","number_rating_1,0","number_rating_1,5","number_rating_2,0","number_rating_2,5","number_rating_3,0","number_rating_3,5","number_rating_4,0","number_rating_4,5","number_rating_5,0",list_of_scores,bayesian_score
0,3210,3,0,0,0,0,0,0,0,0,0,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2.809975
1,4821,2,0,0,0,0,0,0,0,0,0,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0]",3.058070
2,115713,3,0,0,0,0,0,0,0,0,0,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2.809975
3,296,8,0,0,0,0,0,0,0,0,0,"[8, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2.108031
4,2136,2,0,0,0,0,0,0,0,0,0,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0]",3.058070
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27940,2677,0,0,0,0,0,0,0,0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 2]",4.558070
27941,3093,0,0,0,0,0,0,0,0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 2]",4.558070
27942,5957,0,0,0,0,0,0,0,0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 2]",4.558070
27943,8643,0,0,0,0,0,0,0,0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 2]",4.558070


In [95]:
bayesian_score_pd["bayesian_score"] = bayesian_score_pd["list_of_scores"].apply(lambda x: bayesian_score(x))

In [103]:
bayesian_score_pd[["bayesian_score", "movie_id"]].sort_values(by="bayesian_score", ascending=False)[]

,bayesian_score,movie_id
25258,9.844979,296
25411,9.835856,356
26403,9.767409,2959
26668,9.758788,260
26439,9.754233,318


In [113]:
best_bayesian_score_movies = bayesian_score_pd[["bayesian_score", "movie_id"]].sort_values(by="bayesian_score", ascending=False).head(10)["movie_id"].to_list()

# Best bayesian scored movies

In [114]:
graph_mdt.select("movie_id", "title").distinct().filter(F.col("movie_id").isin(best_bayesian_score_movies)).show(truncate=False)

+--------+-----------------------------------------------------+
|movie_id|title                                                |
+--------+-----------------------------------------------------+
|593     |Silence of the Lambs, The (1991)                     |
|1196    |Star Wars: Episode V - The Empire Strikes Back (1980)|
|2959    |Fight Club (1999)                                    |
|318     |Shawshank Redemption, The (1994)                     |
|260     |Star Wars: Episode IV - A New Hope (1977)            |
|110     |Braveheart (1995)                                    |
|79132   |Inception (2010)                                     |
|296     |Pulp Fiction (1994)                                  |
|1197    |Princess Bride, The (1987)                           |
|356     |Forrest Gump (1994)                                  |
+--------+-----------------------------------------------------+



### Neo4j queries

In [ ]:
similarity_query = (
    """MATCH (u1:User)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(u2:User)
    WITH u1, u2, COLLECT({rating1: toFloat(r1.rating), rating2: toFloat(r2.rating)}) AS ratings
    WHERE SIZE(ratings) > 1  // Necesitamos al menos dos películas en común
    WITH u1, u2, ratings,
        REDUCE(s = 0.0, x IN ratings | s + x.rating1 * x.rating2) AS dotProduct,
        REDUCE(s = 0.0, x IN ratings | s + x.rating1) AS ratingSum1,
        REDUCE(s = 0.0, x IN ratings | s + x.rating2) AS ratingSum2,
        REDUCE(s = 0.0, x IN ratings | s + x.rating1 * x.rating1) AS ratingSqSum1,
        REDUCE(s = 0.0, x IN ratings | s + x.rating2 * x.rating2) AS ratingSqSum2,
        SIZE(ratings) AS numRatings
    WITH u1, u2,
        dotProduct,
        ratingSum1, ratingSum2,
        ratingSqSum1, ratingSqSum2,
        numRatings,
        (dotProduct - (ratingSum1 * ratingSum2 / numRatings)) AS numerator,
        (sqrt(ratingSqSum1 - ratingSum1 * ratingSum1 / numRatings) * sqrt(ratingSqSum2 - ratingSum2 * ratingSum2 / numRatings)) AS denominator
    WHERE denominator <> 0
    MERGE (u1)-[s:SIMILARITY]-(u2)
    SET s.pearsonSimilarity = numerator / denominator"""
)

In [16]:
user_id = ""

collaborative_filtering_query = (
    """MATCH (u:User {user_id: """ + user_id + """})-[s:SIMILARITY]-(other:User)-[r:RATED]->(m:Movie)
    WHERE NOT EXISTS { MATCH (u)-[:RATED]->(m) }
    WITH m, other, toFloat(r.rating) AS otherRating, s.pearsonSimilarity AS similarity
    WITH m, AVG(otherRating * similarity) AS weightedRating, SUM(similarity) AS sumSimilarity
    RETURN m.movie_id, sumSimilarity
    LIMIT 10;"""
)

In [ ]:
ontology_filter_query = (
    """MATCH (u:User {user_id: """ + user_id + """})-[:RATED]->(likedMovie:Movie)
    MATCH (likedMovie)-[:BELONGS_TO]->(genre:Genre)<-[:BELONGS_TO]-(recommendedMovie:Movie)
    MATCH (recommendedMovie)<-[:DIRECTED]-(director:Director)
    WHERE NOT (u)-[:RATED]->(recommendedMovie)
    RETURN DISTINCT recommendedMovie.title AS recommendedMovie, COLLECT(DISTINCT genre.name) AS genres, COLLECT(DISTINCT director.name) AS directors"""
   
)